<img src="images/econ151.png" width="200" />

<h1>Project data example</h1>
<h2>Incarceration and earnings data in the NLSY97</h2>

The National Longitudinal Study of Youth 1997 (NLSY97) is a panel survey with repeated interviews of a group of Americans born in the early 1980s. Here is a quote from the [documentation page at the U.S. Bureau of Labor Statistics](https://www.bls.gov/nls/nlsy97.htm):

>    The NLSY97 consists of a nationally representative sample of 8,984 men and women born during the years 1980 through 1984 and living in the United States at the time of the initial survey in 1997.  Participants were ages 12 to 16 as of December 31, 1996.  Interviews were conducted annually from 1997 to 2011 and biennially since then.  The ongoing cohort has been surveyed 18 times as of date.  Data are available from Round 1 (1997-98) through Round 19 (2019-20). 

The NLSY97 cohort includes "supplemental samples" or oversamples of Black and Hispanic youth, in order to understand experiences among those smaller groups better. The overall size of the panel is about 9,000 people who identified as either men (51%) or women (49%).

The NLSY97 measures incarceration, which is not typically something that Census products ask about. [This issue of NLS Investigator](https://www.bls.gov/nls/additional-publications/news-letter-discontinued/release-146.pdf) from 2011 discusses some of the contours of the data on incarceration. 

In [ ]:
library(haven)
library(tidyverse)
library(readxl)

Behind the scenes, I have downloaded data from primarily the 2019 wave using [NLS Investigator](https://www.nlsinfo.org/investigator/pages/login) and cleaned it up using Stata. Most of the variables were already cleaned up, but their variable names were not particularly mnemonic. Some variables included negative numbers where respondents did not reply in the 2019 wave.

In [ ]:
nlsy97_2019 <- read_dta("nlsy97_2019.dta")

In [ ]:
head(nlsy97_2019)

Here is an Excel version of the same data, only with full data labels in many cases rather than values. I will use the Stata version below, but including this code here will mean that the XLSX file will be pulled into your account on datahub, and you can access it directly, for example if you would rather work in Excel or Sheets.

In [ ]:
nlsy97_2019x <- read_excel("nlsy97_2019.xlsx")

In [ ]:
head(nlsy97_2019x)

Here are the contents in greater detail:
````{verbatim}
incarc_tot_months Total months ever incarcerated in 2019
incarc_curr       Currently incarcerated in 2019? Yes = 1, No = 0
pubid             pubid - yth id code 1997
gender            key!sex (symbol) 1997
bmonth		    key!bdate m/y (symbol) 1997
byear             key!bdate m/y (symbol) 1997
cv_sample_type    cv_sample_type 1997
raceth            key!race_ethnicity (symbol) 1997
cv_esr_2006       2006 BLS employment status
age_2019          age at 2019 interview
u4282300          total income: wages, salary, past yr in 2019
female            female? Yes = 1, No = 0
blacknh           Is Black non-Hispanic Yes = 1, No = 0
hispanic          Is Hispanic Yes = 1, No = 0
othernh           Is neither Black nor Hispanic Yes = 1, No = 0
jobinc_2018       Received any income in 2018 from a job such as wages, salary, etc. Yes = 1, No = 0
incwage_2018      Total income from wages, salary, etc. in 2018 before taxes
logincwage_2018   Log of wage/salary income in 2018
edyrs             Z9083800, Years of education/schooling in 2019
incarc_tot_years  Total years ever incarcerated in 2019 (months/12)
````

Sometimes __R__ returns things in unfortunate formats. Here is an option that helps for the next command:

In [ ]:
options(scipen=0)

It turns out that there are people in the sample who are currently incarcerated, and some report earnings in 2018. It might be useful to remove them from the analysis sample.

In [ ]:
nlsy97_2019 <- subset(nlsy97_2019, incarc_curr == 0)

The call below estimates a simple log earnings regression that includes all people not currently incarcerated in 2019 who reported strictly positive earnings in 2018. The omitted category is males who are neither Black nor Hispanic who were 34 years old at the time of interview in 2019. The basic log earnings regression is:

$$
\log Y_i = \alpha + \sum \beta_a \ D_a + \beta_B \ Black_i + \beta_H \ Hispanic_i + \beta_f \ female_i + \beta_e \ edyrs_i + \epsilon_i
$$

where the $D_a$ are indicators for single years of age in 2019 and the other variables are self-explanatory.

In [ ]:
reg1 <- lm(logincwage_2018 ~ factor(age_2019) + blacknh + hispanic + female + edyrs,
           data = nlsy97_2019)
summary(reg1)

(It might be worthwhile to look at just men, since women's earnings are significantly lower than men's. But for now I will just move ahead with the sample that includes both genders.)

Here is a simple extension of the log earnings regression, where we can also control for cumulative total years of incarceration, a measure like years of education or like age but obviously quite different qualitatively.

In [ ]:
reg2 <- lm(logincwage_2018 ~ factor(age_2019) + blacknh + hispanic + female + edyrs
           + incarc_tot_years,
           data = nlsy97_2019)
summary(reg2)

It might also be useful to look at a model where age is linear rather than controlled with indicator variables:

In [ ]:
reg3 <- lm(logincwage_2018 ~ age_2019 + blacknh + hispanic + female + edyrs
           + incarc_tot_years,
           data = nlsy97_2019)
summary(reg3)

It is a little tricky to think about what age really means here, when it only varies between age 34 and 40. But it is probably reasonable to view it the same way we normally would in a regression with more variation in age: it represents, holding other things equal, another (potential) year of work experience. The coefficients on $age$, $edyrs$, and $incarc\_tot\_years$ are all significant and have interesting magnitudes here. 

<hr>

What do you find interesting here? Are there other tests you think would be interesting to run?

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>